# AB_Test_Randomization_Case
## Hao Wu
## Description
Channel KLMN is based in Los Angeles and airs a weekly national political talk show creatively called “US Politics This Week”. Their only TV commercial advertising “US Politics This Week” features the Mayor of Los Angeles. A data scientist at Channel KLMN found that a large fraction of people who saw the commercial for “US Politics This Week” and watched the show are from Los Angeles, while a much lower fraction of people from other cities who saw the commercial watched the show. They then brought this to the attention of the team that produces “US Politics This Week”.

The Executive Producer of “US Politics This Week” suggested that they make TV commercials tailored to some of the biggest cities in the US that feature the Mayors of those cities (i.e., a commercial to be shown in New York City that features the Mayor of New York City, a commercial to be shown in Chicago that features the Mayor of Chicago, etc.). Channel KLMN decided to produce these new commercials and then aired the new commercials in their respective cities, in addition to airing the old commercial (featuring the Mayor of Los Angeles) in all cities as well.

After running the experiment, the data scientist was surprised to find that the test is negative. That is, a lower fraction of people who saw the commercials with their local Mayor watched “US Politics This Week” as compared to people who saw the commercial with the Mayor of Los Angeles!
### Your job is to:
* Reproduce the negative result found above. Is it actually negative?
* Explain what might be happening. Are the commercials with local Mayors really driving a lower fraction of people to watch the show?
* If you found something wrong with the experiment, design an algorithm that returns FALSE if the problem happens again in the future and TRUE if everything is good and the results can be trusted. If you didn’t find anything wrong with the experiment, what is your recommendation to the Executive Producer of “US Politics This Week” regarding whether or not they should continue airing the new commercials?
### Data:
#### test_data:
* viewer_id: the ID of the viewer
* date: the date the viewer saw a commercial for “US Politics This Week”
* tv_make: the make (i.e., brand) of TV
* tv_size: the size of the TV in inches (approximately measured as the diagonal of the screen)
* uhd_capable: whether the TV is (1) or is not (0) capable of displaying Ultra-High-Definition television content
* tv_provider: the cable or satellite TV provider
* total_time_watched: the total amount of TV watched (in hours) on the day in the ‘date’ column
* watched: whether the viewer watched (1) “US Politics This Week” or not (0)
* test: viewers are split into test (1) and control (0) groups; test viewers saw the new commercial with their local Mayor while control viewers saw the old commercial with the Mayor of Los Angeles
#### viewer_data columns:
* viewer_id: the ID of the viewer; same ID as in the test_data file
* gender: the viewer’s gender
* age: the viewer’s age
* city: the viewer’s city

In [1]:
#import packages and data
import pandas as pd
import numpy as np
from scipy import stats
import warnings
from sklearn.tree import DecisionTreeClassifier
warnings.filterwarnings('ignore')

In [2]:
#Input data and basic check
viewer=pd.read_csv("/Users/haowu/Google Drive/Data Science/takehome/Samba_TV/viewer_data.csv")
test=pd.read_csv("/Users/haowu/Google Drive/Data Science/takehome/Samba_TV/test_data.csv")
print("viewer table")
print(viewer.head())
print("test table")
print(test.head())

viewer table
   viewer_id  gender  age      city
0    1918165  Female   39    Dallas
1   27662619  Female   28  New York
2    5493662  Female   53   Detroit
3   14441247    Male   41  New York
4   25595927    Male   53   Seattle
test table
   viewer_id        date tv_make  tv_size  uhd_capable   tv_provider  \
0   24726768  2018-01-16    Sony       70            0       Comcast   
1   25001464  2018-01-18    Sony       32            0           NaN   
2   28291998  2018-01-18    Sony       50            1  Dish Network   
3   17057157  2018-01-19    Sony       32            0       Comcast   
4   29504447  2018-01-17    Sony       32            0       Comcast   

   total_time_watched  watched  test  
0               10.75        0     1  
1                2.75        0     0  
2               20.00        0     0  
3                1.50        0     0  
4               17.50        0     0  


In [3]:
#check users in two table
print(len(viewer.viewer_id.unique())==len(viewer)) #check if the viewer table is unique at viewer_id level
print(len(test.viewer_id.unique())==len(test)) #check if the test table is unique at viewer_id level
print(len(test.viewer_id.unique())-len(viewer.viewer_id.unique())) #check if two tables are 1-to-1 match on the reviewer id

True
False
556


In [4]:
print("viewer table information")
viewer.describe(include="all")


viewer table information


,viewer_id,gender,age,city
count,4.174640e+05,417464,417464.000000,417464
unique,NaN,2,NaN,15
top,NaN,Female,NaN,New York
freq,NaN,209416,NaN,69893
mean,1.837908e+07,NaN,40.473298,NaN
std,1.247548e+07,NaN,12.056198,NaN
min,1.000000e+04,NaN,18.000000,NaN
25%,7.571130e+06,NaN,31.000000,NaN
50%,1.841588e+07,NaN,39.000000,NaN
75%,2.917900e+07,NaN,48.000000,NaN


In [5]:
print("test table information")
print(test.describe(include="all"))

test table information
           viewer_id        date tv_make        tv_size    uhd_capable  \
count   4.180260e+05      418026  418026  418026.000000  418026.000000   
unique           NaN           5       4            NaN            NaN   
top              NaN  2018-01-15    Sony            NaN            NaN   
freq             NaN       86641  271922            NaN            NaN   
mean    1.835585e+07         NaN     NaN      51.874111       0.200858   
std     1.248317e+07         NaN     NaN      12.225413       0.400643   
min     1.000000e+04         NaN     NaN      32.000000       0.000000   
25%     7.526723e+06         NaN     NaN      40.000000       0.000000   
50%     1.838327e+07         NaN     NaN      55.000000       0.000000   
75%     2.916353e+07         NaN     NaN      65.000000       0.000000   
max     3.999992e+07         NaN     NaN      70.000000       1.000000   

       tv_provider  total_time_watched        watched           test  
count       36530

## Insight and Action
As we can see the viewer table includes the unique viewer information, and in the test table, one viewer might have multiple actions. And some viewers in the test table do not have information in the viewer table. For the further analysis, I will combine two tables together by keeping all users in the test table.

In [6]:
data=test.merge(viewer,on="viewer_id",how="left")
print(data.shape)
data.describe(include="all")

(418026, 12)


,viewer_id,date,tv_make,tv_size,uhd_capable,tv_provider,total_time_watched,watched,test,gender,age,city
count,4.180260e+05,418026,418026,418026.000000,418026.000000,365306,418026.000000,418026.000000,418026.000000,417470,417470.000000,417470
unique,NaN,5,4,NaN,NaN,5,NaN,NaN,NaN,2,NaN,15
top,NaN,2018-01-15,Sony,NaN,NaN,Comcast,NaN,NaN,NaN,Female,NaN,New York
freq,NaN,86641,271922,NaN,NaN,109796,NaN,NaN,NaN,209420,NaN,69894
mean,1.835585e+07,NaN,NaN,51.874111,0.200858,NaN,10.039700,0.054547,0.488790,NaN,40.473234,NaN
std,1.248317e+07,NaN,NaN,12.225413,0.400643,NaN,6.179722,0.227094,0.499875,NaN,12.056215,NaN
min,1.000000e+04,NaN,NaN,32.000000,0.000000,NaN,0.250000,0.000000,0.000000,NaN,18.000000,NaN
25%,7.526723e+06,NaN,NaN,40.000000,0.000000,NaN,5.000000,0.000000,0.000000,NaN,31.000000,NaN
50%,1.838327e+07,NaN,NaN,55.000000,0.000000,NaN,9.500000,0.000000,0.000000,NaN,39.000000,NaN
75%,2.916353e+07,NaN,NaN,65.000000,0.000000,NaN,14.500000,0.000000,1.000000,NaN,48.000000,NaN


# Step 1 Replicate A/B Test Result
I will remove the Los Angeles data in the test table, because based on the information, the commercial will not change in the LA area. Therefore, there is no experiment there. 

In [7]:
#define a test function presenting the T test result
def test_result(df,label,group_var='test'):
    
    test = stats.ttest_ind(df.loc[df[group_var] == 1,label], 
                           df.loc[df[group_var] == 0,label], 
                       equal_var=False)
  
    print("statistics")
    print(test.statistic)
    print(" ")
    print("p-value")
    print(test.pvalue)

In [8]:
test_data=data.loc[data.city!="Los Angeles"]
print(test_data.shape)
#Make sure the test and control are correctly split. 
print("test control group size compare",sum(test_data.test==1)/len(test_data),sum(test_data.test==0)/len(test_data))

(365513, 12)
test control group size compare 0.5590143168642429 0.44098568313575714


In [9]:
#check the overall test and control group assignment
test_data.test.value_counts()/len(test_data)

1    0.559014
0    0.440986
Name: test, dtype: float64

In [10]:
test_result(test_data,label="watched")

statistics
-5.720411142093989
 
p-value
1.0635551786694859e-08


Generally, the test and control groups are evenly split. As we can see the watch rate in the test group is 4.58%, and the control group is 4.99%, and the test group is lower by 9%.  This result is not as expected. I will use statistical analysis to check two check following two parts: 
   * Was the experiment set up correctly, i.e. were test/group viewers are ramdonly selected in different segment.


# Step 2 Identify the Potential Reasons

**Actions** define a function applying the DT model to check if the users are fully randomly assigned. the feature importance will output. If the the randomization is correctly implemented, the feature importance of all segments should be low. 

In [11]:
def feature_importance_detect(df,segment_var):       
    model_data=pd.get_dummies(df[segment_var]).fillna(0)
    model=DecisionTreeClassifier(class_weight="balanced",min_impurity_decrease = 0.001)
    model.fit(model_data,df["test"])
    fi = pd.DataFrame({'feature': model_data.columns,
                       'importance': model.feature_importances_}).sort_values('importance', ascending = False)
    return fi

In [12]:
#check the original data
#because the total_time_watched could not be practical in the user segements, so drop it out. 
f_im=feature_importance_detect(test_data,
                               ['tv_make','tv_size','tv_provider','gender','age','city','uhd_capable'])

In [13]:
#list out the segmentation vars over the threshold of feature importance
f_im.loc[~(abs(f_im['importance']-0)<1e-06)]

,feature,importance
23,city_Philadelphia,0.581394
26,city_Seattle,0.418606


In [14]:
#Present the test/control group ratios to confirm the model output
msk=test_data.city.isin(["Philadelphia","Seattle"])
grp=test_data.loc[msk].groupby("city")["test"].agg(['value_counts'])
grp=grp.reset_index("city").join(data.test.value_counts())
grp.columns=["city",'group_count',"total_group_count"]
grp["percent"]=grp["group_count"]/grp["total_group_count"]
grp["group"]=np.where(grp.index==0,"Control","Test")
grp.sort_values(by='city')

,city,group_count,total_group_count,percent,group
0,Philadelphia,4261,213699,0.019939,Control
1,Philadelphia,24495,204327,0.119881,Test
0,Seattle,2846,213699,0.013318,Control
1,Seattle,15850,204327,0.077572,Test


# Insights
Randomization works good in the groups of tv providers and tv makers, because the ratios of control to test are very statble. However, we do see the outiliers in the city group, which are Philadelphia and Seattle, which means, most viewers are assgined in the test groups. 

# Actions
Will Increase the weight of users from Philadelphia and Seattle in the control group so that they are perfectly balanced between the two groups. 


* Figure out how many new rows having test = 0 and city = Philadelphia or Seattle you would need to add to the original dataset to balance the relative frequencies

* Randomly sample from the original dataset those rows and append them to the dataset

* Check that the proportion of users from those two cities is now the same for test and control 

In [15]:
#Use the test/control group ratio to calculate the resampling size. 
sample_dict=dict()
sample_dict={i: grp.loc[(grp.group=='Test') & (grp["city"]==i) ,'percent'].values[0]\
             /grp.loc[(grp.group=='Control') & (grp["city"]==i) , 'percent'].values[0]
                        for i in set(grp["city"])}
sample_dict

{'Seattle': 5.824667008126684, 'Philadelphia': 6.012327662070761}

In [16]:
#Here we will use the oversampling approach, so will resample the control groups of AR and UR with replacement. 
over_sample_data=[test_data]

for c in sample_dict.keys():
    sub_df=test_data.loc[(test_data.city==c)&(test_data.test==0)]
    sub_df=sub_df.sample(frac=sample_dict[c], 
                         replace=True, random_state=1)
    print(sub_df.shape)
    over_sample_data.append(sub_df)
over_sample_data=pd.concat(over_sample_data,ignore_index=True)

(16577, 12)
(25619, 12)


In [17]:
#recheck the test/control groups of those two cities
msk=over_sample_data.city.isin(["Philadelphia","Seattle"])
grp=over_sample_data.loc[msk].groupby("city")["test"].agg(['value_counts'])
grp=grp.reset_index("city").join(over_sample_data.test.value_counts())
grp.columns=["city",'group_count',"total_group_count"]
grp["percent"]=grp["group_count"]/grp["total_group_count"]
grp["group"]=np.where(grp.index==0,"Control","Test")
grp.sort_values(by='city')

,city,group_count,total_group_count,percent,group
0,Philadelphia,29880,203382,0.146916,Control
1,Philadelphia,24495,204327,0.119881,Test
0,Seattle,19423,203382,0.095500,Control
1,Seattle,15850,204327,0.077572,Test


In [18]:
#To make sure the resampling doesn't cause the inbalances among other segments, do the Decesion Tree check again
resample_f_im=feature_importance_detect(over_sample_data,
                               ['tv_make','tv_size','tv_provider','gender','age','city','uhd_capable'])
#check if there is any features having significant importance. We can see no feature has significance in the group
#allocation. 
resample_f_im.loc[~(abs(resample_f_im['importance']-0)<1e-06)]

,feature,importance


In [19]:
#reproduce the test results
test_result(over_sample_data,label='watched')

statistics
1.5610131287673206
 
p-value
0.11852142940765997


# Conclusion

* As using the random sampling approaches, the unbalanced randomization was fixed. However the test result is not statistically significant, with p-value above 0.1. Therefore the experiment didn't improve the watch rate. If there is no other metrics to consider, I would not recommend to launch “US Politics This Week”  national wide. 


In [20]:
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook AB_Test_Randomization_Case.ipynb to html',
 '[NbConvertApp] Writing 339660 bytes to AB_Test_Randomization_Case.html']